# COURSEWORK 2


#### Datasets

The dataset analyzed in this coursework contains daily weather observations from numerous Australian weather stations. The daily observations are available from http://www.bom.gov.au/climate/data. (Copyright Commonwealth of Australia 2010, Bureau of Meteorology).

This notebook is already run, but in order to  re run this notebook (if needed) "weatherAUS.csv" should be loaded.

This notebook throws "AUSdf.csv": a dataset with more than 3 million rows. "AUSdf.csv" was the dataset used in PySpark. And the results of the codes for that dataset are in files attached.



#### Instructions

This contains two files:

1. The Notebook (PART I)

2. Spark Codes and results (PART II)

Read first this Notebook and proceed with  Spark Code modelling report.



#### Workflow

1. Exploratory analysis and preprocessing:

In this process I will mainly work in Python (on Jupypter Notebook) with null value procedures and dummification of categorical values.

2. Multiplicating data

After cleaning the dataset. I will append the same cleaned dataset several times in order to simulate an enormous size dataset (of about 3.6 Million rows and 63 columns)

3. Deploying simple regression models

Two simple regression models will be done to predict numerical values, since this would take many hour to do in Python this will be done in Spark.



### ¿What was modelled?

1. MinTemp: minimun temperature in degrees celsius in day 

Metric --> RMSE
Model --> simple regression with all predictors

2. Rainfall: The amount of rainfall recorded for the day in mm

Metric --> RMSE
Model --> simple regression with all predictors


###  Variables

MinTemp: The minimum temperature in degrees celsius

MaxTemp: The maximum temperature in degrees celsius

Rainfall: The amount of rainfall recorded for the day in mm

Evaporation: The so-called Class A pan evaporation (mm) in the 24 hours to 9am

Sunshine: The number of hours of bright sunshine in the day.

WindGustDir: The direction of the strongest wind gust in the 24 hours to midnight

WindGustSpeed: The speed (km/h) of the strongest wind gust in the 24 hours to midnight

WindDir9am: Direction of the wind at 9am

WindDir3pm: Direction of the wind at 3pm

WindSpeed9am: Wind speed (km/hr) averaged over 10 minutes prior to 9am

WindSpeed3pm: Wind speed (km/hr) averaged over 10 minutes prior to 3pm

Humidity9am: Humidity (percent) at 9am

Humidity3pm: Humidity (percent) at 3pm

Pressure9am: Atmospheric pressure (hpa) reduced to mean sea level at 9am

Pressure3pm: Atmospheric pressure (hpa) reduced to mean sea level at 3pm

Cloud9am: Fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast.

Cloud3pm: Fraction of sky obscured by cloud (in "oktas": eighths) at 3pm. See Cload9am for a description of the values

Temp9am: Temperature (degrees C) at 9am

Temp3pm: Temperature (degrees C) at 3pm



### Technologies used 

Python (local) and Pyspark (dsm)


#### Notes

This Coursework is extremely simple, I would have like to work with different algorithms and with categorical models but unfortunately I wasn't able to succeed many days of trying to learn Spark (but I will still be involved trying to learn more from Big Data application in my carrer!). 

My coursework is largely based on lab 9.

Originally I was working with the same type of dataset (for Brazil), but it was too big for me to clean it in Python  or in the cluster, so I chose this one, but after cleaning it I made it bigger by concatenating its rows several times.




In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('weatherAUS.csv')

In [ ]:
data.head()

The following columns have more than 25% missing values, so they will be dropped

In [65]:
data = data.drop(columns=['Cloud9am','Cloud3pm', 'Evaporation', 'Sunshine','RISK_MM', 'Location'])

Dropping any row with null values

In [69]:
data = data.dropna()
data.isnull().any()

Date             False
MinTemp          False
MaxTemp          False
Rainfall         False
WindGustDir      False
WindGustSpeed    False
WindDir9am       False
WindDir3pm       False
WindSpeed9am     False
WindSpeed3pm     False
Humidity9am      False
Humidity3pm      False
Pressure9am      False
Pressure3pm      False
Temp9am          False
Temp3pm          False
RainToday        False
RainTomorrow     False
dtype: bool

### Dummification

In [70]:
data["RainToday"] = data["RainToday"].map({"No":0, "Yes":1})
data["RainTomorrow"] = data["RainTomorrow"].map({"No":0, "Yes":1})

In [71]:
data.shape

(112925, 18)

In [79]:
categorical_variables = ["WindGustDir", "WindDir9am", "WindDir3pm"]

In [ ]:
data = pd.get_dummies(data, columns=categorical_variables)

In [81]:
data.shape

(112925, 63)

In [28]:
data.describe()

MinTemp        MaxTemp       Rainfall  WindGustSpeed  \
count  112925.000000  112925.000000  112925.000000  112925.000000   
mean       12.664721      23.655670       2.377892      40.786611   
std         6.254135       6.982702       8.602968      13.321774   
min        -8.200000       2.600000       0.000000       7.000000   
25%         8.100000      18.300000       0.000000      31.000000   
50%        12.400000      23.100000       0.000000      39.000000   
75%        17.200000      28.700000       0.800000      48.000000   
max        33.900000      48.100000     367.600000     135.000000   

        WindSpeed9am   WindSpeed3pm    Humidity9am    Humidity3pm  \
count  112925.000000  112925.000000  112925.000000  112925.000000   
mean       15.179163      19.500545      67.404162      50.669055   
std         8.344304       8.580097      18.911610      20.768550   
min         2.000000       2.000000       0.000000       0.000000   
25%         9.000000      13.000000      56.000000      36.000000   
50%        13.000000      19.000000      68.000000      51.000000   
75%        20.000000      24.000000      81.000000      65.000000   
max        87.000000      87.000000     100.000000     100.000000   

         Pressure9am    Pressure3pm       ...        WindDir3pm_NNW  \
count  112925.000000  112925.000000       ...         112925.000000   
mean     1017.425847    1015.048962       ...              0.053673   
std         7.032654       6.957822       ...              0.225372   
min       980.500000     977.100000       ...              0.000000   
25%      1012.800000    1010.300000       ...              0.000000   
50%      1017.400000    1015.000000       ...              0.000000   
75%      1022.100000    1019.700000       ...              0.000000   
max      1041.000000    1039.600000       ...              1.000000   

       WindDir3pm_NW   WindDir3pm_S  WindDir3pm_SE  WindDir3pm_SSE  \
count  112925.000000  112925.000000  112925.000000   112925.000000   
mean        0.056533       0.072048       0.073721        0.069019   
std         0.230949       0.258568       0.261318        0.253488   
min         0.000000       0.000000       0.000000        0.000000   
25%         0.000000       0.000000       0.000000        0.000000   
50%         0.000000       0.000000       0.000000        0.000000   
75%         0.000000       0.000000       0.000000        0.000000   
max         1.000000       1.000000       1.000000        1.000000   

       WindDir3pm_SSW  WindDir3pm_SW   WindDir3pm_W  WindDir3pm_WNW  \
count   112925.000000  112925.000000  112925.000000   112925.000000   
mean         0.059225       0.067797       0.070383        0.061395   
std          0.236047       0.251399       0.255793        0.240054   
min          0.000000       0.000000       0.000000        0.000000   
25%          0.000000       0.000000       0.000000        0.000000   
50%          0.000000       0.000000       0.000000        0.000000   
75%          0.000000       0.000000       0.000000        0.000000   
max          1.000000       1.000000       1.000000        1.000000   

       WindDir3pm_WSW  
count   112925.000000  
mean         0.068824  
std          0.253156  
min          0.000000  
25%          0.000000  
50%          0.000000  
75%          0.000000  
max          1.000000  

[8 rows x 62 columns]

### Multiplicating data by making copies to simulate big volume

In [82]:
data

Date  MinTemp  MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  \
0       2008-12-01     13.4     22.9       0.6           44.0          20.0   
1       2008-12-02      7.4     25.1       0.0           44.0           4.0   
2       2008-12-03     12.9     25.7       0.0           46.0          19.0   
3       2008-12-04      9.2     28.0       0.0           24.0          11.0   
4       2008-12-05     17.5     32.3       1.0           41.0           7.0   
5       2008-12-06     14.6     29.7       0.2           56.0          19.0   
6       2008-12-07     14.3     25.0       0.0           50.0          20.0   
7       2008-12-08      7.7     26.7       0.0           35.0           6.0   
8       2008-12-09      9.7     31.9       0.0           80.0           7.0   
9       2008-12-10     13.1     30.1       1.4           28.0          15.0   
10      2008-12-11     13.4     30.4       0.0           30.0          17.0   
11      2008-12-12     15.9     21.7       2.2           31.0          15.0   
12      2008-12-13     15.9     18.6      15.6           61.0          28.0   
13      2008-12-14     12.6     21.0       3.6           44.0          24.0   
15      2008-12-17     14.1     20.9       0.0           22.0          11.0   
16      2008-12-18     13.5     22.9      16.8           63.0           6.0   
17      2008-12-19     11.2     22.5      10.6           43.0          24.0   
18      2008-12-20      9.8     25.6       0.0           26.0          17.0   
19      2008-12-21     11.5     29.3       0.0           24.0           9.0   
20      2008-12-22     17.1     33.0       0.0           43.0          17.0   
21      2008-12-23     20.5     31.8       0.0           41.0          19.0   
22      2008-12-24     15.3     30.9       0.0           33.0           6.0   
23      2008-12-25     12.6     32.4       0.0           43.0           4.0   
24      2008-12-26     16.2     33.9       0.0           35.0           9.0   
26      2008-12-28     20.1     32.7       0.0           48.0          13.0   
27      2008-12-29     19.7     27.2       0.0           46.0          19.0   
28      2008-12-30     12.5     24.2       1.2           50.0          11.0   
29      2008-12-31     12.0     24.4       0.8           39.0          17.0   
30      2009-01-01     11.3     26.5       0.0           56.0          19.0   
31      2009-01-02      9.6     23.9       0.0           41.0          19.0   
...            ...      ...      ...       ...            ...           ...   
142162  2017-05-25     14.6     26.3       0.0           37.0          19.0   
142164  2017-05-27      9.3     28.0       0.0           28.0           6.0   
142165  2017-05-28      8.0     24.6       0.0           33.0          11.0   
142166  2017-05-29     12.7     22.2       0.0           37.0          19.0   
142167  2017-05-30      9.4     22.7       0.0           35.0          13.0   
142168  2017-05-31      5.4     20.5       0.0           46.0          20.0   
142169  2017-06-01      5.6     19.4       0.0           43.0          22.0   
142170  2017-06-02      1.5     19.2       0.0           37.0          17.0   
142171  2017-06-03      3.8     19.8       0.0           39.0          19.0   
142172  2017-06-04      4.5     18.8       0.0           31.0          13.0   
142173  2017-06-05      4.9     20.7       0.0           37.0          17.0   
142174  2017-06-06      1.2     20.8       0.0           43.0          13.0   
142175  2017-06-07      0.5     21.7       0.0           43.0          13.0   
142176  2017-06-08      4.0     20.0       0.0           39.0          13.0   
142177  2017-06-09      3.4     19.8       0.0           43.0          19.0   
142178  2017-06-10      3.2     18.6       0.0           46.0          15.0   
142179  2017-06-11      2.4     19.1       0.0           33.0          17.0   
142180  2017-06-12      5.1     19.7       0.0           46.0          13.0   
142181  2017-06-13      2.5     21.2       0.0           35.0   

In [83]:
data1 = data

In [84]:
data2 = pd.concat([data, data1], ignore_index=True)

In [85]:
data3 = data2

In [86]:
data4 = pd.concat([data3, data2], ignore_index=True)

In [87]:
data5 = data4

In [88]:
data6 = pd.concat([data4, data5], ignore_index=True) 

In [90]:
data7 = data6

In [91]:
data_aus = pd.concat([data6, data7], ignore_index=True)

In [94]:
data8 = data_aus

### This is the dataset that will be used for the models!

In [95]:
df_aus = pd.concat([data8, data_aus], ignore_index=True)

In [96]:
df_aus.shape

(3613600, 63)

In [93]:
data_aus.describe()

MinTemp       MaxTemp      Rainfall  WindGustSpeed  WindSpeed9am  \
count  1.806800e+06  1.806800e+06  1.806800e+06   1.806800e+06  1.806800e+06   
mean   1.266472e+01  2.365567e+01  2.377892e+00   4.078661e+01  1.517916e+01   
std    6.254109e+00  6.982673e+00  8.602932e+00   1.332172e+01  8.344269e+00   
min   -8.200000e+00  2.600000e+00  0.000000e+00   7.000000e+00  2.000000e+00   
25%    8.100000e+00  1.830000e+01  0.000000e+00   3.100000e+01  9.000000e+00   
50%    1.240000e+01  2.310000e+01  0.000000e+00   3.900000e+01  1.300000e+01   
75%    1.720000e+01  2.870000e+01  8.000000e-01   4.800000e+01  2.000000e+01   
max    3.390000e+01  4.810000e+01  3.676000e+02   1.350000e+02  8.700000e+01   

       WindSpeed3pm   Humidity9am   Humidity3pm   Pressure9am   Pressure3pm  \
count  1.806800e+06  1.806800e+06  1.806800e+06  1.806800e+06  1.806800e+06   
mean   1.950054e+01  6.740416e+01  5.066905e+01  1.017426e+03  1.015049e+03   
std    8.580062e+00  1.891153e+01  2.076846e+01  7.032625e+00  6.957793e+00   
min    2.000000e+00  0.000000e+00  0.000000e+00  9.805000e+02  9.771000e+02   
25%    1.300000e+01  5.600000e+01  3.600000e+01  1.012800e+03  1.010300e+03   
50%    1.900000e+01  6.800000e+01  5.100000e+01  1.017400e+03  1.015000e+03   
75%    2.400000e+01  8.100000e+01  6.500000e+01  1.022100e+03  1.019700e+03   
max    8.700000e+01  1.000000e+02  1.000000e+02  1.041000e+03  1.039600e+03   

            ...        WindDir3pm_NNW  WindDir3pm_NW  WindDir3pm_S  \
count       ...          1.806800e+06   1.806800e+06  1.806800e+06   
mean        ...          5.367279e-02   5.653310e-02  7.204782e-02   
std         ...          2.253709e-01   2.309483e-01  2.585671e-01   
min         ...          0.000000e+00   0.000000e+00  0.000000e+00   
25%         ...          0.000000e+00   0.000000e+00  0.000000e+00   
50%         ...          0.000000e+00   0.000000e+00  0.000000e+00   
75%         ...          0.000000e+00   0.000000e+00  0.000000e+00   
max         ...          1.000000e+00   1.000000e+00  1.000000e+00   

       WindDir3pm_SE  WindDir3pm_SSE  WindDir3pm_SSW  WindDir3pm_SW  \
count   1.806800e+06    1.806800e+06    1.806800e+06   1.806800e+06   
mean    7.372150e-02    6.901926e-02    5.922515e-02   6.779721e-02   
std     2.613172e-01    2.534870e-01    2.360457e-01   2.513977e-01   
min     0.000000e+00    0.000000e+00    0.000000e+00   0.000000e+00   
25%     0.000000e+00    0.000000e+00    0.000000e+00   0.000000e+00   
50%     0.000000e+00    0.000000e+00    0.000000e+00   0.000000e+00   
75%     0.000000e+00    0.000000e+00    0.000000e+00   0.000000e+00   
max     1.000000e+00    1.000000e+00    1.000000e+00   1.000000e+00   

       WindDir3pm_W  WindDir3pm_WNW  WindDir3pm_WSW  
count  1.806800e+06    1.806800e+06    1.806800e+06  
mean   7.038300e-02    6.139473e-02    6.882444e-02  
std    2.557915e-01    2.400530e-01    2.531554e-01  
min    0.000000e+00    0.000000e+00    0.000000e+00  
25%    0.000000e+00    0.000000e+00    0.000000e+00  
50%    0.000000e+00    0.000000e+00    0.000000e+00  
75%    0.000000e+00    0.000000e+00    0.000000e+00  
max    1.000000e+00    1.000000e+00    1.000000e+00  

[8 rows x 62 columns]

#### This dataset will be used in the dsm cluster

In [97]:
df_aus.to_csv("AUSdf.csv")